In [6]:
import pandas as pd
import numpy as np

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, auc, accuracy_score, mean_squared_error

In [2]:
num_submission = 1 
# Load data
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')

In [7]:
#features = [c for c in train_df.columns if c not in ['ID_code', 'target']]
features = train_df.columns.drop(['ID_code', 'target']).tolist()

In [8]:
train_features_df = train_df[features]
target_series = train_df['target']

In [9]:
train_values_df, holdout_test_df, train_target_series, holdout_test_target_series = train_test_split(
    train_features_df,
    target_series,
    test_size=0.2,
    random_state=0
)

In [4]:
params = {
    'task': 'train', 'max_depth': -1, 'boosting_type': 'gbdt',
    'objective': 'binary', 'num_leaves': 3, 'learning_rate': 0.01,
}

In [5]:
y_hat = 0.0
predictions = {}
scores = {}
for feature in features: # loop over all features
    lgb_train = lgb.Dataset(train_values_df[feature].values.reshape(-1,1), train_target_series)
    gbm = lgb.train(params, lgb_train, 110, verbose_eval=5)
    prediction = gbm.predict(holdout_test_df[feature].values.reshape(-1,1), num_iteration=gbm.best_iteration)
    scores[feature] = roc_auc_score(holdout_test_target_series, prediction)
    predictions[feature] = prediction
    y_hat += prediction

In [ ]:
#y_hat /= len(features)

In [10]:
#sub = pd.DataFrame({"ID_code": test_df.ID_code.values})
#sub["target"] = y_hat
#sub.to_csv('submission{}.csv'.format(num_submission), index=False)

In [12]:
auc?